# **Introduction:**

This file serves to host an attempted Keras implementation of an Adaptive Neuro-Fuzzy Inference System (ANFIS).

**Date Created:** 22/01/2025

**Date Modified:** 28/01/2025

# **Import Packages:**

This section imports all the necessary packages for the ANFIS implementation.

In [43]:
# import packages:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os
from pickle import dump
from itertools import product
from keras.layers import Layer
from tensorflow.keras import Input, Model

# **Function & Layer Definition:**

This section creates the necessary custom functions and layers for this ANFIS implementation within Keras. 

In [44]:
# need to first define the initial layer -> the membership function layer:
class MembershipFunctionLayer(Layer):
    # constructor:
    def __init__(self, num_inputs, num_mfs, params = None, **kwargs):   # by including **kwargs, we allow for additional arguments from keras, like name or dtype
        super(MembershipFunctionLayer, self).__init__(**kwargs)         # we are subclassing from the keras layer -> telling the constructor to make our layer like a keras layer
        self.num_inputs = num_inputs            # define the number of inputs to the ANFIS 
        self.num_mfs = num_mfs                  # define the number of membership functions per input
        self.num_rules = num_mfs ** num_inputs  # the number of rules is calculated as such:

        # next is the initialization of the antecedent parameters:
        if params is not None:
            # initialize custom parameters defined by the user:
            self.mf_params = self.add_weight(
                shape=(self.num_inputs, self.num_mfs, 3),       # define their shape, (num_inputs, num_mfs, 3) as we have 3 params for a triangular mf
                initializer=tf.constant_initializer(params),    # initialize as constants from the provided array
                trainable=True,                                 # set to trainable
                name="Antecedent Params",                       # assign them a name
            )
            print('Custom parameters have been set.')
        else:
            # initialize raw membership parameters:
            raw_params = self.add_weight(
                shape = (self.num_inputs, self.num_mfs, 3),                                 # define their shape, (num_inputs, num_mfs, 3) 
                initializer = tf.keras.initializers.RandomUniform(-1.0, 1.0, seed = 1234),  # initialize as a random, uniform distribution
                trainable = True,                                                           # set to trainable
                name = "Raw Antecedent Params",                                             # assign them a name 
            )

            # sort the parameters such that a <= b <= c:
            sorted_params = tf.sort(raw_params, axis = 1)           # sort such that a <= b <= c
            self.mf_params = self.add_weight(                       
                shape = (self.num_inputs, self.num_mfs, 3),                     # set the shape: (num_inputs, num_mfs, 3) as we have 3 params for a triangular mf
                initializer = tf.constant_initializer(sorted_params.numpy()),   # initialize as the sorted array of params
                trainable = True,                                               # set to trainable
                name = 'Antecedent Params'                                      # assign them a name
            )
            print('Random parameters have been set.')
 
    # function for visualizing the membership functions:
    def plot_mf(self, max_values, mf_names = None):
        # if the user did not provide names:
        if mf_names is None:
            mf_names = [f'MF {i + 1}' for i in range(self.num_mfs)]
        
        # make sure that the number of names matches the number of membership functions:
        if len(mf_names) != self.num_mfs:
            raise ValueError(f'Expected {self.num_mfs} membership functions, but got {len(mf_names)} instead.')
        
        # make sure that the provided max values match the number of membership functions:
        if len(max_values) != self.num_mfs:
            raise ValueError(f'Expected {self.num_mfs} max values, but got {len(max_values)} instead.') 
        
        # create linspace based on max values:
        input_range = {}
        for i in range(self.num_inputs):
            input_range[i] = np.linspace(0, max_values[i], 1000)

        # plot the mfs:
        for input_index in range(self.num_inputs):
            x_values = input_range[input_index]
            plt.figure(figsize = (12,8))

            # plot each mf for the selected input:
            for i in range(self.num_mfs):
                params = self.mf_params[input_index, i].numpy()
                a, b, c = params
                y_values = [np.maximum(0.0, np.minimum((x - a) / (b - a + 1e-6), (c - x) / (c - b + 1e-6))) for x in x_values]
                
                plt.plot(x_values, y_values, label = f'{mf_names[i]}')
            plt.title(f'Membership Functions for Input X{input_index + 1}')
            plt.xlabel('Input Value')
            plt.ylabel('Degree of Membership')
            plt.legend()
            plt.grid(True)
        plt.show()
    
    # need to define the call -> this is what gets executed by the layer:
    def call(self, inputs):
        # initialize list to hold membership values:
        membership_values = []

        # loop through each input:
        for i in range(self.num_inputs):
            # extract membership functions for the current input:
            input_mf_params = self.mf_params[i]  # symbolic tensor

            # compute membership values for all MFs for the current input:
            mf_values = []

            for j in range(self.num_mfs):
                a = input_mf_params[j, 0]
                b = input_mf_params[j, 1]
                c = input_mf_params[j, 2]

                mf_value = tf.maximum(
                    0.0,
                    tf.minimum(
                        (inputs[:, i] - a) / (b - a + 1e-6),
                        (c - inputs[:, i]) / (c - b + 1e-6),
                    ),
                )
                mf_values.append(mf_value)

            # stack MFs for this input
            membership_values.append(tf.stack(mf_values, axis = -1))

        # stack all membership values (shape: batch_size, num_inputs, num_mfs)
        return tf.stack(membership_values, axis = 1)

# need to now define the second layer -> the firing strength layer:
class FiringStrengthLayer(Layer):
    # constructor:
    def __init__(self, num_inputs, num_mfs, **kwargs):
        super(FiringStrengthLayer, self).__init__(**kwargs)
        self.num_inputs = num_inputs
        self.num_mfs = num_mfs
        self.num_rules = num_mfs ** num_inputs

    # call function:
    def call(self, membership_values):
        # get batch size:
        batch_size = tf.shape(membership_values)[0]  

        # initialize firing strengths
        firing_strengths = tf.ones((batch_size, self.num_rules), dtype = tf.float32)    

        # generate all rule combinations:
        rules = list(product(range(self.num_mfs), repeat = self.num_inputs))  # example: [(0, 0, 0), (0, 0, 1), ...]

        # need to check each input, each mf combination, and multiply their values together:
        for rule_index, combination in enumerate(rules):
            # print(f'rule: {rule_index + 1} | combination: {combination}')
            rule_strength = tf.ones((batch_size, ), dtype = tf.float32)

            for input_index, mf_index in enumerate(combination):
                # print(f'input: {input_index + 1} | mf: {mf_index + 1}')

                # correctly extract the fuzzified values based on the combination index:
                rule_strength *= membership_values[:, input_index, mf_index]
                # print(f'strength: {rule_strength}')
            
            # update the firing strengths:
            rule_strength = tf.expand_dims(rule_strength, axis = -1)  # shape: (batch_size, 1)
            firing_strengths = tf.concat(
                [firing_strengths[:, :rule_index], rule_strength, firing_strengths[:, rule_index + 1:]],
                axis = 1,
            )

        return firing_strengths

# need to now define the third layer -> the normalization layer:
class NormalizationLayer(Layer):
    # constructor:
    def __init__(self, num_inputs, num_mfs, **kwargs):
        super(NormalizationLayer, self).__init__(**kwargs)
        self.num_inputs = num_inputs
        self.num_mfs = num_mfs
        self.num_rules = num_mfs ** num_inputs

    # call function:
    def call(self, firing_strengths):
        # get batch size:
        batch_size = tf.shape(firing_strengths)[0]

        # get total firing strength:
        total_strength = tf.reduce_sum(firing_strengths, axis = 1, keepdims = True)
        # print(f'total strength: {total_strength}')
        
        # normalize the firing strengths:
        normalized_strengths = firing_strengths / (total_strength + 1e-10)

        return normalized_strengths

# need to now define the fourth layer -> the consequent layer:
class ConsequentLayer(Layer):
    # constructor:
    def __init__(self, num_inputs, num_mfs, **kwargs):
        super(ConsequentLayer, self).__init__(**kwargs)
        self.num_inputs = num_inputs
        self.num_mfs = num_mfs
        self.num_rules = num_mfs ** num_inputs

        # need to also initialize the consequent parameters:
        self.consequent_params = self.add_weight(
            shape = (self.num_rules, self.num_inputs + 1),
            initializer = tf.keras.initializers.RandomUniform(-1.0, 1.0, seed = 1234),
            trainable = True,
            name = 'Consequent Params'
        )

    # call function:
    def call(self, input_list):
        # inputs are a list -> extract values:
        normalized_strengths, inputs = input_list

        # get the batch size
        batch_size = tf.shape(normalized_strengths)[0]

        # add bias term to inputs: shape (batch_size, num_inputs + 1)
        inputs_with_bias = tf.concat([inputs, tf.ones((batch_size, 1), dtype=tf.float32)], axis = -1)

        # reshape normalized_strengths to (batch_size, num_rules, 1)
        normalized_strengths = tf.reshape(normalized_strengths, (batch_size, self.num_rules, 1))

        # get consequent parameters: shape (num_rules, num_inputs + 1)
        consequent_params = self.consequent_params  # already initialized as a weight

        # expand inputs_with_bias to match the rule axis: (batch_size, num_rules, num_inputs + 1)
        inputs_with_bias_expanded = tf.expand_dims(inputs_with_bias, axis = 1)

        # calculate the consequent for each rule
        consequents = tf.reduce_sum(normalized_strengths * inputs_with_bias_expanded * consequent_params, axis = 2)

        return consequents

# now can define the final layer -> the output layer:
class OutputLayer(Layer):
    # constructor:
    def __init__(self, num_inputs, num_mfs, **kwargs):
        super(OutputLayer, self).__init__(**kwargs)
        self.num_inputs = num_inputs
        self.num_mfs = num_mfs
        self.num_rules = num_mfs ** num_inputs

    # call function:
    def call(self, consequents):
        output = tf.reduce_sum(consequents, axis = 1, keepdims = True)
        return output

# define a custom function for building models:
def BuildAnfis(input_shape, num_inputs, num_mfs, antecedent_params = None):
    # define the inputs:
    inputs = Input(shape = input_shape)

    # add the custom layers:
    membership_layer = MembershipFunctionLayer(num_inputs = num_inputs, num_mfs = num_mfs, params = antecedent_params)(inputs)
    firing_layer = FiringStrengthLayer(num_inputs = num_inputs, num_mfs = num_mfs)(membership_layer)
    normalization_layer = NormalizationLayer(num_inputs = num_inputs, num_mfs = num_mfs)(firing_layer)
    consequent_layer = ConsequentLayer(num_inputs = num_inputs, num_mfs = num_mfs)([normalization_layer, inputs])
    output_layer = OutputLayer(num_inputs = num_inputs, num_mfs = num_mfs)(consequent_layer)

    # create and compile the model:
    model = Model(inputs = inputs, outputs = output_layer)
    model.compile(optimizer = 'adam', loss = 'mse')

    print('Model compiled!')

    return model


# **Model Creation:**

This section creates a given model and explores its parameters and structure.

In [45]:
# define the following to be used in model generation:
num_inputs = 3
num_mfs = 3
max_values = np.array([10, 25, 50])
mf_names = ['Low', 'Medium', 'High']
params = np.array([
    [  # Parameters for input 1
        [0, 0, 6],
        [5/6, 5, 55/6],
        [4, 10, 10]
    ],
    [  # Parameters for input 2
        [0, 0 , 15],
        [25/12, 12.5, 275/12],
        [10, 25, 25]
    ],
    [  # Parameters for input 3
        [0, 0, 30],
        [25/6, 25, 275/6],
        [15, 50, 50]
    ]
])

# # define inputs:
# inputs = tf.constant([[2, 9, 21], [8, 23, 48]], dtype=tf.float32)

# make model:
model = BuildAnfis(input_shape = (3,), num_inputs = num_inputs, num_mfs = num_mfs, antecedent_params = params)

Custom parameters have been set.
Model compiled!


Explore the model structure:

In [46]:
model.summary(show_trainable = True)

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)      ┃ Output Shape    ┃   Param # ┃ Connected to   ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_4     │ (None, 3)       │         0 │ -              │   -   │
│ (InputLayer)      │                 │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ membership_funct… │ (None, 3, 3)    │        27 │ input_layer_4… │   Y   │
│ (MembershipFunct… │                 │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ firing_strength_… │ (None, 27)      │         0 │ membership_fu… │   -   │
│ (FiringStrengthL… │                 │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ normalization_la… │ (None, 27)      │         0 │ firing_streng… │   -   │
│ (NormalizationLa… │                 │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ consequent_layer… │ (None, 27)      │       108 │ normalization… │   Y   │
│ (ConsequentLayer) │                 │           │ input_layer_4… │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ output_layer_4    │ (None, 1)       │         0 │ consequent_la… │   -   │
│ (OutputLayer)     │                 │           │                │       │
└───────────────────┴─────────────────┴───────────┴────────────────┴───────┘

 Total params: 135 (540.00 B)

 Trainable params: 135 (540.00 B)

 Non-trainable params: 0 (0.00 B)

# **Data Import & Processing:**

Need to now import the required data and split it into training, validation, and testing. From this, the model can be trained and its performance verified.

In [47]:
# get the data path:
files_in_dir = os.listdir(os.getcwd())
data_path = os.path.join(os.getcwd(), files_in_dir[files_in_dir.index('V3_Data.csv')])

# load data based on data path:
df = pd.read_csv(data_path)
print('Data successfully loaded')

Data successfully loaded


Need to first split into features and labels:

In [48]:
# split into X and Y:
x_data = df.drop(columns = 'Suitability')
y_data = df['Suitability']

Now the data can be split into training, validation, and testing sets:

In [49]:
# split the data using train_test_split:
x_train, x_filler, y_train, y_filler = train_test_split(x_data, y_data, test_size = 0.2)
x_val, x_test, y_val, y_test = train_test_split(x_filler, y_filler, test_size = 0.5)

# get the split results:
print(f'There are {x_train.shape[0]} training examples')
print(f'There are {x_val.shape[0]} validation examples')
print(f'There are {x_test.shape[0]} testing examples\n')

# extract the input shape:
INPUT_SHAPE = x_data.shape[1]
print(f'Input shape is of form: {INPUT_SHAPE}')

There are 8000 training examples
There are 1000 validation examples
There are 1000 testing examples

Input shape is of form: 3


Now we can scale the data. The data must be scaled using the same scaler used on the training set, to ensure that the model is consistent:

In [50]:
# define a scaler:
scaler = StandardScaler()

# scale each set:
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

# save the scaler:
dump(scaler, open('scaler.pkl', 'wb'))